In [1]:
from itertools import permutations
import math
from scipy.optimize import fsolve, minimize

def replace_zeroes(i):
    if i == 0: 
        return -1
    return i

def replace_min_1(i):
    if i == -1:
        return 0
    return int(i)

def generate_permutations(N, fracs):
    arr = []
    for i in range(len(fracs)):
        arr.extend([i] * int(N*fracs[i])) # add fracs proportion of a species
    arr = [replace_zeroes(i) for i in arr]
    perms = permutations(arr, len(arr))
    return list(set(list(perms)))

generate_permutations(5, [0.4, 0.6])

[(1, 1, -1, 1, -1),
 (1, 1, 1, -1, -1),
 (1, -1, -1, 1, 1),
 (1, -1, 1, -1, 1),
 (-1, 1, 1, -1, 1),
 (-1, 1, -1, 1, 1),
 (1, -1, 1, 1, -1),
 (-1, 1, 1, 1, -1),
 (1, 1, -1, -1, 1),
 (-1, -1, 1, 1, 1)]

In [18]:
def dot(s1, s2):
    return int(s1 * s2)

def energy(perm, J, h):
    e = 0
    for i in range(len(perm)):
        cur = replace_min_1(perm[i])
        nex = replace_min_1(perm[(i+1)%len(perm)])
        j = J[cur][nex]
        hi = h[cur]
        
        e += j * dot(perm[i], perm[(i+1)%len(perm)])
        e += hi * perm[i]
    
    return e

def prob(perm, J, h):
    return math.exp(-energy(perm, J, h))
    
def entropy(J, h, permutations):
    pr = [prob(perm, J, h) for perm in permutations]
    tot = sum(pr)
    pr = [i/tot for i in pr]
    ent = -sum([i*math.log(i) for i in pr])
    return ent

In [19]:
L = 2
N = 5
fracs = [0.4, 0.6] 
J = [[0.5, 0.5], [0.5, 0.5]]
h = [0.25] * 2
perms = generate_permutations(N, fracs)

print("Permutations:", perms)
print("Energy of {}: {}".format(perms[0], energy(perms[0], J, h)))
print("Probability of {}: {}".format(perms[0], prob(perms[0], J, h)))
print("Entropy:", entropy(J, h, perms))

Permutations: [(1, 1, -1, 1, -1), (1, 1, 1, -1, -1), (1, -1, -1, 1, 1), (1, -1, 1, -1, 1), (-1, 1, 1, -1, 1), (-1, 1, -1, 1, 1), (1, -1, 1, 1, -1), (-1, 1, 1, 1, -1), (1, 1, -1, -1, 1), (-1, -1, 1, 1, 1)]
Energy of (1, 1, -1, 1, -1): -1.25
Probability of (1, 1, -1, 1, -1): 3.4903429574618414
Entropy: 1.974771767521308


In [4]:
def ent(x, perms, L):
    h = x[:L]
    J = x[1:]
    J = [J[i*L:(i+1)*L] for i in range(L)]
    return -entropy(J, h, perms)
    
x = [0.25] * 2
x.extend([0.5]*4)

m = minimize(ent, x, args=(perms, 2))
print(m)

      fun: -3.321928094887361
 hess_inv: array([[ 1.00000000e+00,  1.13872522e-06,  1.13872522e-06,
         1.13872522e-06,  1.13869402e-06,  0.00000000e+00],
       [ 1.13872522e-06,  9.23292794e-01, -7.67037903e-02,
        -7.67037903e-02, -7.67062609e-02,  0.00000000e+00],
       [ 1.13872522e-06, -7.67037903e-02,  9.23299626e-01,
        -7.67003741e-02, -7.67028448e-02,  0.00000000e+00],
       [ 1.13872522e-06, -7.67037903e-02, -7.67003741e-02,
         9.23299626e-01, -7.67028448e-02,  0.00000000e+00],
       [ 1.13869402e-06, -7.67062609e-02, -7.67028448e-02,
        -7.67028448e-02,  9.23294685e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])
      jac: array([0.00000000e+00, 2.98023224e-08, 2.98023224e-08, 2.98023224e-08,
       2.98023224e-08, 0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 48
      nit: 4
     njev: 6
   status: 0
  success: True
  

In [5]:
h = m.x[:L]
J = m.x[L:]
J = [J[i*L:(i+1)*L] for i in range(L)]

print(h, J)

pr = [prob(perm, J, h) for perm in perms]
print("\n", pr)

[ 0.24999997 -0.18749995] [array([0.06249996, 0.06249996]), array([0.06250012, 0.5       ])]

 [2.2535347280780735, 1.1331483628614263, 1.1331483628614263, 2.2535347280780735, 2.2535347280780735, 2.253534728078074, 2.2535347280780735, 1.133148362861426, 1.133148362861426, 1.133148362861426]


In [6]:
def constr(x, perms, W, a1, a2):
    h = m.x[:L]
    J = m.x[L:]
    J = [J[i*L:(i+1)*L] for i in range(L)]
    avgs = [[0, 0], [0, 0]]
    gl_avgs = [[0, 0], [0, 0]]
    
    pr = [prob(perm, J, h) for perm in perms]
    tot = sum(pr)
    pr = [i/tot for i in pr]
    
    for p in range(len(perms)):
        perm = perms[p]
        for i in range(len(perm)):
            cur, nex = perm[i], perm[(i+1)%len(perm)]
            if cur == -1:
                if nex == -1:
                    avgs[0][0] += dot(cur, nex)
                else:
                    avgs[0][1] += dot(cur, nex)
            else:
                if nex == -1:
                    avgs[1][0] += dot(cur, nex)
                else:
                    avgs[1][1] += dot(cur, nex)
        for i in range(2):
            for j in range(2):
                gl_avgs[i][j] += avgs[i][j] * pr[p]
                    
    con = [[0, 0], [0, 0]]
    for i in range(2):
        for j in range(2):
            gl_avgs[i][j] /= len(perms[0])
            con[i][j] += gl_avgs[i][j] - W[i][j]
            
    return con[a1][a2]

def create_constr_dict(perms, W, L):
    return [{'type': 'eq', 'fun': constr, 'args': (perms, W, x1, x2)} for x1 in range(L) for x2 in range(L)]

In [7]:
W = [[1, 1], [1, 1]]
print(constr(perms, W, J, 1, 1))
di = create_constr_dict(perms, W, L)
print(di)

1.0
[{'type': 'eq', 'fun': <function constr at 0x7fc587be1d90>, 'args': ([(1, 1, -1, 1, -1), (1, 1, 1, -1, -1), (1, -1, -1, 1, 1), (1, -1, 1, -1, 1), (-1, 1, 1, -1, 1), (-1, 1, -1, 1, 1), (1, -1, 1, 1, -1), (-1, 1, 1, 1, -1), (1, 1, -1, -1, 1), (-1, -1, 1, 1, 1)], [[1, 1], [1, 1]], 0, 0)}, {'type': 'eq', 'fun': <function constr at 0x7fc587be1d90>, 'args': ([(1, 1, -1, 1, -1), (1, 1, 1, -1, -1), (1, -1, -1, 1, 1), (1, -1, 1, -1, 1), (-1, 1, 1, -1, 1), (-1, 1, -1, 1, 1), (1, -1, 1, 1, -1), (-1, 1, 1, 1, -1), (1, 1, -1, -1, 1), (-1, -1, 1, 1, 1)], [[1, 1], [1, 1]], 0, 1)}, {'type': 'eq', 'fun': <function constr at 0x7fc587be1d90>, 'args': ([(1, 1, -1, 1, -1), (1, 1, 1, -1, -1), (1, -1, -1, 1, 1), (1, -1, 1, -1, 1), (-1, 1, 1, -1, 1), (-1, 1, -1, 1, 1), (1, -1, 1, 1, -1), (-1, 1, 1, 1, -1), (1, 1, -1, -1, 1), (-1, -1, 1, 1, 1)], [[1, 1], [1, 1]], 1, 0)}, {'type': 'eq', 'fun': <function constr at 0x7fc587be1d90>, 'args': ([(1, 1, -1, 1, -1), (1, 1, 1, -1, -1), (1, -1, -1, 1, 1), (1, -1, 1, 

In [8]:
x = [0.25] * 2
x.extend([0.5]*4)

m = minimize(ent, x, args=(perms, 2), constraints=di, options={'maxiter':10000, 'disp': True})
print(m)

Singular matrix C in LSQ subproblem    (Exit mode 6)
            Current function value: -2.9268598593862314
            Iterations: 1
            Function evaluations: 8
            Gradient evaluations: 1
     fun: -2.9268598593862314
     jac: array([0.        , 0.31844047, 0.31844047, 0.31844047, 0.31844047,
       0.        ])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 8
     nit: 1
    njev: 1
  status: 6
 success: False
       x: array([0.25, 0.25, 0.5 , 0.5 , 0.5 , 0.5 ])


In [9]:
def constr(x, perms, W): # , a1, a2):
    h = x[:L]
    J = x[L:]
    J = [J[i*L:(i+1)*L] for i in range(L)]
    avgs = [[0, 0], [0, 0]]
    gl_avgs = [[0, 0], [0, 0]]
    
    for perm in perms:
        for i in range(len(perm)):
            cur, nex = perm[i], perm[(i+1)%len(perm)]
            if cur == -1:
                if nex == -1:
                    avgs[0][0] += dot(cur, nex)
                else:
                    avgs[0][1] += dot(cur, nex)
            else:
                if nex == -1:
                    avgs[1][0] += dot(cur, nex)
                else:
                    avgs[1][1] += dot(cur, nex)
        for i in range(2):
            for j in range(2):
                gl_avgs[i][j] += avgs[i][j] * prob(perm, J, h)
                    
    con = [[0, 0], [0, 0]]
    for i in range(2):
        for j in range(2):
            gl_avgs[i][j] /= len(perms[0])
            con[i][j] += gl_avgs[i][j] - W[i][j]
            
    return con # con[a1][a2]

def lagrange(x, perms, L, W, Lmd):
    entropy = ent(x, perms, L)
    con = constr(x, perms, W)
    for i in range(2):
        for j in range(2):
            entropy += con[i][j]**2 * Lmd[i][j]
    return entropy

Lmd = [[3]*2 for _ in range(2)]
print(Lmd, "\n")
m = minimize(lagrange, x, args=(perms, 2, W, Lmd), options={'maxiter':1000, 'disp': True})
print(m)

[[3, 3], [3, 3]] 

Optimization terminated successfully.
         Current function value: 8.678075
         Iterations: 33
         Function evaluations: 272
         Gradient evaluations: 34
      fun: 8.678074779862879
 hess_inv: array([[  93844.57615894,  151754.06704494, -113154.09689428,
         -19299.91721025,  -19299.91709736, -452656.451356  ],
       [ 151754.06704494,  245400.06566702, -182980.42014027,
         -31209.20625151,  -31209.20606896, -731986.48078785],
       [-113154.09689428, -182980.42014027,  136438.87889422,
          23270.99656865,   23270.99643254,  545800.20780249],
       [ -19299.91721025,  -31209.20625151,   23270.99656865,
           3969.89865031,    3968.89862709,   93092.61021347],
       [ -19299.91709736,  -31209.20606896,   23270.99643254,
           3968.89862709,    3969.89860387,   93092.60966896],
       [-452656.451356  , -731986.48078785,  545800.20780249,
          93092.61021347,   93092.60966896, 2183395.47719692]])
      jac: array(

In [10]:
h = m.x[:L]
J = m.x[L:]
J = [J[i*L:(i+1)*L] for i in range(L)]

print(h, J)

pr = [(perm, prob(perm, J, h)) for perm in perms]
for p in pr:
    print(p)

[-11.96922001 -18.84088618] [array([14.42731212,  2.2067873 ]), array([ 2.20678682, 57.96415166])]
((1, 1, -1, 1, -1), 6.474658267681952e-08)
((1, 1, 1, -1, -1), 2.852414973068351e-41)
((1, -1, -1, 1, 1), 2.85241497306831e-41)
((1, -1, 1, -1, 1), 6.474658267681952e-08)
((-1, 1, 1, -1, 1), 6.474658267681952e-08)
((-1, 1, -1, 1, 1), 6.474658267681952e-08)
((1, -1, 1, 1, -1), 6.474658267681883e-08)
((-1, 1, 1, 1, -1), 2.852414973068351e-41)
((1, 1, -1, -1, 1), 2.852414973068351e-41)
((-1, -1, 1, 1, 1), 2.852414973068351e-41)


In [11]:
con = constr(m.x, perms, W)
print(con)

[[-0.9999998964054677, -1.0000004920740284], [-1.0000004920740284, -0.9999995985711874]]
